In [5]:
import os
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch import nn, optim
import matplotlib.pyplot as plt
import numpy as np
import random
import shutil 
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
from torchvision import models


In [6]:
from pathlib import Path

train_dir = Path("data/brain_tumor_dataset/Training")
test_dir = Path("data/brain_tumor_dataset/Testing")


In [8]:
for class_folder in train_dir.iterdir():
    num_images = len(list(class_folder.glob("*.jpg")))
    print(f"{class_folder.name}: {num_images} images")

pituitary: 1457 images
.DS_Store: 0 images
notumor: 1595 images
glioma: 1321 images
meningioma: 1339 images


In [9]:
for class_folder in test_dir.iterdir():
    num_images = len(list(class_folder.glob("*.jpg")))
    print(f"{class_folder.name}: {num_images} images")

pituitary: 300 images
notumor: 405 images
glioma: 300 images
meningioma: 306 images


In [10]:
from pathlib import Path

source_dir = Path("data/brain_tumor_dataset/Training")
target_dir = Path("data/clean_split")

train_target = target_dir / "train"
val_target = target_dir / "val"

train_target.mkdir(parents=True, exist_ok=True)
val_target.mkdir(parents=True, exist_ok=True)


In [11]:
for class_folder in source_dir.iterdir():
    if class_folder.is_dir():
        train_class_dir = train_target / class_folder.name
        val_class_dir = val_target / class_folder.name
        
        train_class_dir.mkdir(parents=True, exist_ok=True)
        val_class_dir.mkdir(parents=True, exist_ok=True)

         # List all images
        image_paths = list(class_folder.glob("*.jpg"))
        
        # Shuffle
        random.shuffle(image_paths)
        
        # Split 80/20
        split_idx = int(len(image_paths) * 0.8)
        train_images = image_paths[:split_idx]
        val_images = image_paths[split_idx:]
        
        for img_path in train_images:
            shutil.copy(img_path, train_class_dir / img_path.name)
        
        for img_path in val_images:
            shutil.copy(img_path, val_class_dir / img_path.name)


In [12]:
test_source_dir = Path("data/brain_tumor_dataset/Testing")
test_target_dir = Path("data/clean_split/test")

for class_folder in test_source_dir.iterdir():
    if class_folder.is_dir():
        target_class_dir = test_target_dir / class_folder.name
        target_class_dir.mkdir(parents=True, exist_ok=True)
        
        for img_path in class_folder.glob("*.jpg"):
            shutil.copy(img_path, target_class_dir / img_path.name)


In [25]:
train_dataset = ImageFolder(root="data/clean_split/train", transform=transform)
val_dataset = ImageFolder(root="data/clean_split/val", transform=transform)
test_dataset = ImageFolder(root="data/clean_split/test", transform=transform)


In [26]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [15]:
resnet = models.resnet18(pretrained=True)
resnet.fc = nn.Linear(in_features=512, out_features=4)


/Users/nilufersevdeozdemir/Desktop/compmedprojects/brain-mri-classifier/venv/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/nilufersevdeozdemir/Desktop/compmedprojects/brain-mri-classifier/venv/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Move model to device
resnet = resnet.to(device)

Using device: cpu


In [17]:
def train_model(model, train_loader, val_loader, criterion, optimizer, device, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        
        running_loss = 0.0
        correct = 0
        total = 0
        
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100 * correct / total
        
        print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {epoch_loss:.4f} Accuracy: {epoch_acc:.2f}%")

                # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()

                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_epoch_loss = val_loss / len(val_loader)
        val_epoch_acc = 100 * val_correct / val_total

        print(f"Validation Loss: {val_epoch_loss:.4f} Accuracy: {val_epoch_acc:.2f}%")



In [18]:
import torch.nn as nn

criterion = nn.CrossEntropyLoss()


In [19]:
import torch.optim as optim

optimizer = optim.Adam(resnet.parameters(), lr=0.001)


In [20]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.parameters(), lr=0.001)


In [22]:
from torchvision import transforms


In [24]:
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # In case images are grayscale but ResNet expects 3 channels
    transforms.Resize((224, 224)),                # Resize to fit ResNet18 input size
    transforms.ToTensor(),                        # Convert PIL image → Tensor
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Normalize to [-1, 1]
])



In [ ]:
num_epochs = 5
train_model(resnet, train_loader, val_loader, criterion, optimizer, device, num_epochs)


Epoch [1/5] Loss: 0.4226 Accuracy: 84.95%
Validation Loss: 0.3950 Accuracy: 85.62%
Epoch [2/5] Loss: 0.2140 Accuracy: 92.29%
Validation Loss: 0.1251 Accuracy: 95.03%
Epoch [3/5] Loss: 0.1597 Accuracy: 94.54%
Validation Loss: 0.1294 Accuracy: 95.46%
Epoch [4/5] Loss: 0.1354 Accuracy: 95.32%
Validation Loss: 0.0745 Accuracy: 97.73%
